In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("trips_2").getOrCreate()

directory = "/home/ubuntu/working/spark-examples/data"
trip_files = "trips/*" 

trips_df = spark.read.csv(f"file://{directory}/{trip_files}", inferSchema=True, header=True)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/14 04:43:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/14 04:44:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
trips_df.printSchema()
# 여기까지는 데이터 레이크, 데이터 웨어하우스

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [3]:
trips_df.createOrReplaceTempView("trips")

In [4]:
# Lake -> Warehouse -> Mart
# Mart 에서 App 만듦
# Warehouse, Mart -> ML(머신러닝)/ 웨어하우스,마트 단계에서 데이터(피쳐, 타겟)를 끌어와서 머신러닝에 활용

운행 거리(`trip_distance`)에 따른 요금(`total_amount`)를 예측

In [5]:
# DataMart 구성
query = """
select trip_distance,
        total_amount
from trips
where total_amount < 5000
    and total_amount > 0
    and passenger_count < 5
    and trip_distance > 0 
    and trip_distance < 500
    and TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    and TO_DATE(tpep_pickup_datetime) < '2021-08-01' 
"""

data_df = spark.sql(query)
data_df.createOrReplaceTempView("data") 

In [6]:
train_df,  test_df = data_df.randomSplit([0.8, 0.2], seed=42)

In [7]:
# 훈련 데이터는 캐싱
train_df.cache()

DataFrame[trip_distance: double, total_amount: double]

In [8]:
train_df.count(), test_df.count()

(10660955, 2665176)

In [9]:
data_df.show()
# Feature -> Matrix(Nx1), 각 데이터가 스칼라가 아닌 벡터여야함, 매트릭스로 변환
# Target -> 벡터O

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
|         12.4|        43.8|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
|         16.2|        45.3|
|         3.58|        19.3|
|         0.91|        14.8|
|         2.57|        12.8|
|          0.4|         5.3|
|         3.26|        17.3|
|        13.41|       47.25|
|         18.3|       61.42|
|         1.53|       14.16|
|          2.0|        11.8|
|         16.6|       54.96|
|         15.5|       56.25|
|          1.3|        16.8|
+-------------+------------+
only showing top 20 rows



`feature`가 하나 밖에 없지만 `VectorAssembler` 를 활용해서 1차원 배열 형식으로 각각의 데이터를 만들어준다.
- 즉 RN => RNx1

In [10]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=['trip_distance'], outputCol="features")
vec_train_df = vec_assembler.transform(train_df)

vec_train_df.show()

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 20 rows



In [11]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    labelCol="total_amount",
    featuresCol="features") #Transformer

In [12]:
# 모델 생성
lr_model = lr.fit(vec_train_df)

23/06/14 04:53:32 WARN Instrumentation: [54eb3214] regParam is zero, which might cause numerical instability and overfitting.
23/06/14 04:53:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/14 04:53:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/06/14 04:54:11 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
# 테스트 데이터로 예측 - transform
vec_test_df = vec_assembler.transform(test_df)
predictions = lr_model.transform(vec_test_df)
predictions.show() 

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874

In [14]:
# Evaluator 를 쓰지 않고 Train 데이터에 대한 RMSE값 확인
lr_model.summary.rootMeanSquaredError

6.308435272654453

In [15]:
lr_model.summary.r2

0.765849929974943

In [16]:
spark.stop()